## Housing Master File

#### This notebook collates and cleans a number of files we recieved from the buildings 
#### department. It outputs a shapefile and a csv of all the building data. 

#### NYU CUSP Small Cities Team - Summer 2017
#### Made with Python 2.7 (anaconda)

##### Input files should all be placed in Raw folder. 
#####
##### "Housing Action Type Code File.xlsx"
##### "Housing Case Type Code Master File.xlsx"
##### "Housing ce300ap Case Action History.csv"
##### "Housing Code Enforcement Case Master File.xlsx"
##### "Housing Location Master File.xlsx"

##### Note* Location Master Ancillary File is not used

In [20]:
import seaborn
import os
import subprocess
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as dt
import time
from shapely.geometry import Point

Data is in Data->Raw


In [21]:
#Set Input/Output path variables.  This accesses data from the Raw Data File
uppath = ".."
dpath = "Data"
inpath = "Raw"
procdatapath = "Processed"
shapepath = "Shapefiles"
outpath = "Interim"

In [22]:
#Set the file Names
fname1 = "Housing Action Type Code File.xlsx"
fname2 = "Housing Case Type Code Master File.xlsx"
fname3 = "Housing ce300ap Case Action History.csv"
fname4 = "Housing Code Enforcement Case Master File.xlsx"
fname5 = "Housing Location Master File.xlsx"

In [23]:
#Set the import paths 
infile1 = os.path.join(uppath, dpath, inpath, fname1)
infile2 = os.path.join(uppath, dpath, inpath, fname2)
infile3 = os.path.join(uppath, dpath, inpath, fname3)
infile4 = os.path.join(uppath, dpath, inpath, fname4)
infile5 = os.path.join(uppath, dpath, inpath, fname5)

In [24]:
#Read the Data Into pandas
action_type_code = pd.read_excel(infile1)
case_type_code = pd.read_excel(infile2)
case_action_history = pd.read_csv(infile3)
code_enforce_case = pd.read_excel(infile4)
location_master = pd.read_excel(infile5)

### Clean the location data

In [25]:
#Function to create address for geocoding on the location master file
def clean_address (row):
    '''
    Make new address column for merging & geocoding
    Return value into new column
    '''
    if pd.isnull(row["Street Dir."]):
        value = str(row["Street#"]) + " " + str(row["Street Name"]) + " Cleveland Heights " + str(row["Zip"])
    else:
        value = str(row["Street#"]) + " " + str(row["Street Dir."]) + " " + str(row["Street Name"]) + " Cleveland Heights " + str(row["Zip"])
    return value

In [26]:
#Apply the function to create address for geocoding and merge operation
location_master["address2"] = location_master.apply(clean_address, axis=1)


In [27]:
#In location master file.  Add old column names to a list. Add new column names 
#to a list.  

old_loc_names = [u'Location ID',               u'Street#',
                 u'Street Dir.',           u'Street Name',
                u'Apartment ID',                   u'Zip',
               u'Carrier Route',           u'Owners Name',
                  u'Owner Type',      u'Owners Address 2',
            u'Owners Address 3',            u'Owners Zip',
        u'Owners Carrier Route',      u'Owners Area Code',
                u'Owners Phone',        u'Land Key Sect1',
              u'Land Key Sect2',        u'Land Key Sect3',
              u'Land Key Sect4',        u'Land Key Sect5',
              u'Land Key Sect6',        u'Land Key Sect7',
              u'Land Key Sect8',        u'Land Key Sect9',
             u'Land Key Sect10',   u'Parcel_Address Flag',
       u'Alternate Location ID',      u'Street Qualifier',
                     u'Acreage',     u'Property Use Code',
        u'Undivided Interest %',         u'User Defined1',
               u'User Defined2',         u'User Defined3',
                    u'address2']

new_loc_names = [u'Location_ID',               u'Street_num',
                 u'Street_Dir.',           u'Street_Name',
                u'Apartment_ID',                   u'Zip',
               u'Carrier_Route',           u'Owners_Name',
                  u'Owner_Type',      u'Owners_Address2',
            u'Owners_Address3',            u'Owners_Zip',
        u'Owners_CarrierRoute',      u'Owners_Area_Code',
                u'Owners_Phone',        u'Land_Key_Sect1',
              u'Land_Key_Sect2',        u'Land_Key_Sect3',
              u'Land_Key_Sect4',        u'Land_Key_Sect5',
              u'Land_Key_Sect6',        u'Land_Key_Sect7',
              u'Land_Key_Sect8',        u'Land_Key_Sect9',
             u'Land_Key_Sect10',   u'Parcel_AddressFlag',
       u'AlternateLocationID',      u'Street_Qualifier',
                     u'Acreage',     u'PropertyUseCode',
        u'Undivided Interest %',         u'UserDefined1',
               u'UserDefined2',         u'UserDefined3',
                    u'address2']

In [28]:
#Rename the columns
location_master.rename(columns=dict(zip(old_loc_names, new_loc_names)), inplace=True)

In [29]:
#set file name
loc_master_clean_excel = "loc_master_clean.xlsx"

In [30]:
loc_master_clean_csv = "loc_master_clean.csv"

In [31]:
#create output path string
loc_master_cleanOUTxl = os.path.join(uppath, dpath, outpath, loc_master_clean_excel)
loc_master_cleanOUTxl

'../Data/Interim/loc_master_clean.xlsx'

In [32]:
loc_master_cleanOUTcsv = os.path.join(uppath, dpath, outpath, loc_master_clean_csv)
loc_master_cleanOUTcsv

'../Data/Interim/loc_master_clean.csv'

In [33]:
#Export to file excel

location_master.to_excel(loc_master_cleanOUTxl)

In [34]:
#export to csv
#location_master.to_csv(loc_master_cleanOUTcsv, encoding='utf-8')

### Merge Case History with the Action Codes

In [35]:
#Merge the Case History onto the Action-type codes. 
case_action_merge = pd.merge(case_action_history, action_type_code, left_on = "Action_Type_Code",\
                             right_on = "Action Typ Code", left_index= True)

In [36]:
#Rename the columns in the new case action merge file
#Old column names in a list
case_old = [u'Case_Year',                 u'Case#',\
                 u'Action_Seq#',      u'Action_Type_Code',\
                 u'Action_Flag', u'Next_Action_#_of_Days',\
               u'Action_Number',       u'Action Typ Code',\
            u'Action Code Desc',           u'Action Flag',\
       u'Action Letter Library',         u'Action Letter',\
           u'Action Letter Doc',  u'Print Violation Flag']

#New column names in a list
case_new = [u'Case_Year',                 u'Case_num',\
                 u'Action_Seq_num',      u'Action_Type_Code',\
                 u'Action_Flag', u'Next_Action_#_of_Days',\
               u'Action_Number',       u'Action_Typ_Code',\
            u'Action_Code_Desc',           u'Action_Flag',\
       u'Action_Letter_Library',         u'Action_Letter',\
           u'Action_Letter_Doc',  u'Print_Violation_Flag']

In [37]:
#Rename columns in the new merged file using the lists. 

case_action_merge.rename(columns=dict(zip(case_old, case_new)), inplace=True)

In [38]:
#Create file names
case_action_mergexl = 'case_action_merge.xlsx'
case_action_mergecsv = 'case_action_merge.csv'


In [39]:
#Create output strings
case_action_mergeOUTxl = os.path.join(uppath, dpath, outpath, case_action_mergexl)
case_action_mergeOUTcsv = os.path.join(uppath, dpath, outpath, case_action_mergecsv)

In [40]:
#Export to file excel

#case_action_merge.to_excel(case_action_mergeOUTxl)

In [41]:
#Export to csv

#case_action_merge.to_csv(case_action_mergeOUTcsv)

### Merge Codes and Code Type

In [42]:
#Merge codes into case type codes

code_enforce_merge = pd.merge(code_enforce_case, case_type_code, left_on = "Case Type Code",\
                             right_on = "Case Type Code", left_index= True)

In [43]:
#rename columns lists

#The original column names
code_enforce_old = [u'Case Yr', u'Case Number', u'Land ID #', u'Case Status',\
       u'Last Inspection #', u'Case Type Code', u'Case Inspector',\
       u'Tenant Name', u'Tenant #', u'Case Origination Code',\
       u'Code Enforcement Board#', u'Case Type Description',\
       u'Responsible Department']

#New columns; the ones we want
code_enforce_old_new = [u'Case_Yr', u'Case_Number', u'Land_ID', u'Case_Status',\
       u'Last_Inspection_num', u'Case_Type_Code', u'Case_Inspector',\
       u'Tenant_Name', u'Tenant_num', u'Case_Origination_Code',\
       u'Code_Enforcement_Boardnum', u'Case_Type_Description',\
       u'Responsible_Department']

In [44]:

#rename columns
code_enforce_merge.rename(columns=dict(zip(code_enforce_old, code_enforce_old_new)), inplace=True)

In [45]:
#Create file names
code_enforce_mergexl = 'code_enforce_merge.xlsx'
code_enforce_mergecsv = 'DD_code_enforce_merge.csv'

In [46]:
#Create output strings
code_enforce_mergeOUTxl = os.path.join(uppath, dpath, outpath, code_enforce_mergexl)
code_enforce_mergeOUTcsv = os.path.join(uppath, dpath, outpath, code_enforce_mergecsv)

In [47]:
#Export to file excel

#code_enforce_merge.to_excel(code_enforce_mergeOUTxl)

In [48]:
#Export to csv

#code_enforce_merge.to_csv(code_enforce_mergeOUTcsv, encoding='utf-8')

### Merge Code description onto Case Actions

In [49]:
#Merge Code description to code actions

code_case_all = pd.merge(code_enforce_merge, case_action_merge, left_on = "Case_Number",\
                             right_on = "Case_num", indicator=True)

### Merge Locations onto all the code Cases

In [50]:
#Merge locations and code cases

location_code_master = pd.merge(location_master, code_case_all, left_on = "Location_ID",\
                             right_on = "Land_ID")

In [51]:
location_code_masterxl = 'location_code_master.xlsx'
location_code_mastercsv = 'location_code_master.csv'

In [52]:
#Create output strings
location_code_masterOUTxl = os.path.join(uppath, dpath, outpath, location_code_masterxl)
location_code_masterOUTcsv = os.path.join(uppath, dpath, outpath, location_code_mastercsv)

In [53]:
#Export to file excel
#uncomment to export

#location_code_master.to_excel(location_code_masterOUTxl)

In [54]:
#Export to csv
#Uncomment to run

#location_code_master.to_csv(location_code_masterOUTcsv, encoding='utf-8')

Bring in Geocoded address and merge

In [55]:
RawgeocodeFile = "House_location_geocodes_All.csv"

In [58]:
#Set file string for import
geofileImport = os.path.join(uppath, dpath, outpath, RawgeocodeFile)


In [59]:
#Import Geocoded address file

all_geocode = pd.read_csv(geofileImport)

In [60]:
#Drop crap columns from Geocoded file

all_geocode.drop('Unnamed: 0', axis = 1, inplace = True)

In [61]:
# Merge Geocodes onto location file using address feild

location_code_final2 = pd.merge(location_code_master, all_geocode.drop_duplicates(subset=['address']),\
                                how = "left", left_on = "address2",\
                             right_on = "address")

In [62]:
#Put uneeded columns in a list

finaldropcols = ['_merge', 'Tenant_num', 'Carrier_Route', 'Owners_CarrierRoute', 'UserDefined3',\
                'UserDefined2', 'UserDefined3', 'address', 'Case_Inspector',\
                 'Responsible_Department', 'Code_Enforcement_Boardnum', 'Land_Key_Sect4', 'Land_Key_Sect5',\
                 'Land_Key_Sect6', 'Land_Key_Sect7', 'Land_Key_Sect8', 'Land_Key_Sect9', 'Land_Key_Sect10',\
                 'AlternateLocationID', 'Owners_Phone',\
                 'Undivided Interest %', 'Acreage', 'Action_Letter']
                 
                 
                 

In [63]:
#More uneeded columns

finaldropcols2 = ['Action_Letter_Library', 'Parcel_AddressFlag', 'Case_num', 'Action_Typ_Code',\
                  'Case_Yr', 'UserDefined1'] 

In [64]:
#Drop columns from first list
location_code_final2.drop(finaldropcols2, axis = 1, inplace = True)

In [65]:
#Drop columns from second
location_code_final2.drop(finaldropcols, axis = 1, inplace = True)

In [66]:
#Drop Nan's from coordinate feild
location_code_final2.dropna(subset=["coordinate"], inplace = True)


In [67]:
#Create Lat and Long column
#Change them to Pointly points
#Equip the DataFrame with shapefile points.
location_code_final2['latitude'] = [float(str(row['coordinate']).split(',')[0][1:]) for ix,row in location_code_final2.iterrows()]
location_code_final2['longitude'] = [float(str(row['coordinate']).split(',')[1][1:-1]) for ix,row in location_code_final2.iterrows()]
geometry1 = gpd.GeoSeries([Point(xy) for xy in zip(location_code_final2.longitude, location_code_final2.latitude)])
location_code_final2 = gpd.GeoDataFrame(location_code_final2, geometry=geometry1)
location_code_final2.crs = {'init' :'epsg:4326'}

In [68]:
#Function to create address for address matchingmatch addresses 
def owner_occupied (row):
    '''
    detemine if a building is owner occupied by comparing two address fields 
    Returns 1 for owner occupied
    Return 0 for not owner occupied
    '''
    #Define actual address
    if pd.isnull(row["Street_Dir."]):
        value1 = str(row["Street_num"]) + " " + str(row["Street_Name"]) + " " + str(row["Zip"])
    else:
        value1 = str(row["Street_num"]) + " " + str(row["Street_Dir."]) + " " + str(row["Street_Name"]) + " " +\
        str(row["Zip"])
    
    #Define Owner Address 
    value2 = str(row["Owners_Address3"]) + " " + str(row["Owners_Zip"])
    
    #If owner zip and zip do not match return 0; if they do compare full string
    if str(row["Zip"]) != str(row["Owners_Zip"]):
        return 0
    #Comapre full String
    if str(value1) == str(value2): 
        return 1
    else:
        return 0 

In [69]:
location_code_final2["own_occupd"] = location_code_final2.apply(owner_occupied, axis=1)

In [70]:
geoFileExport_FileName = "code_viol_Dbase.shp"

In [71]:
#Delete
path34543 = "~/Users/trondheim/Desktop/"

In [72]:
#Delete
path34543

'~/Users/trondheim/Desktop/'

In [73]:
pwd

u'/Users/Danny1/Documents/NYU/SmallCities-Performance-Analysis/Cleveland_Heights/Code'

In [74]:
#Set file string for export
geoFileExport = os.path.join(uppath, dpath, procdatapath, shapepath, geoFileExport_FileName)


In [76]:
#location_code_final2.to_file("../Data/interim/Code_Enfocement.shp")

KeyboardInterrupt: 

In [ ]:
Break Here

In [77]:
csv_export_FileName = "code_viol_Dbase.csv"

In [78]:
csvFileExport = os.path.join(uppath, dpath, outpath, csv_export_FileName)

In [80]:
#location_code_final2.to_csv(csvFileExport)

In [82]:
#Try to export to shapefile
#WOnky with DateTimes
#location_code_final2.to_file(geoFileExport)

### The code above cleans and creates the master database type file.  The code below is supposed to aggregate by address the negative actions per property 

In [83]:
#Place negative action codes in a list

neg_actions = ["V2", "V3", "V4", "V5", "V6", "VR", "N1", "N2", "N3", "NA", "I1", "I2", "CS", "CT", "C1", "C2", "C3", "C4",\
"C5", "C6", "CA", "CB",\
"CC", "CD", "CE", "CF", "CG", "CH", "CI", "CJ", "CK", "CL", "CM", "CN", "CO", "CP", "CQ", "CS", "CT"] 

In [84]:
#Create a mask to only grab negative actions from the master file
neg_mask = location_code_final2["Action_Type_Code"].isin(neg_actions)

In [85]:
#Create a GeoPandas data frame using the negative mask on location master
negative_master = gpd.GeoDataFrame(location_code_final2[neg_mask])


In [97]:
owners = gpd.read_file('../Data/Interim/Master_Building/CH_Only_Clip.shp')
owners = owners[['PARCEL_ID', 'geometry']]

In [105]:
owners.dropna(inplace=True)

In [109]:
code_enforcement = negative_master[['Action_Code_Desc', 'Case_Status', 'Case_Year', 'geometry']]
code_enforcement.crs = {'init' :'epsg:4326'}
code_enforcement.dropna(inplace=True)
code_enforcement = gpd.sjoin(code_enforcement, owners, how='inner', op='intersects')
code_enforcement.drop(['index_right'], axis=1, inplace=True)

/Users/Danny1/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [110]:
code_enforcement.to_csv('../Data/Interim/Neg_Actions.csv')

In [86]:
#Make dictionary to define aggregation

aggro4 = {"Case_Type_Code":lambda x:x.value_counts().index[0],"Case_Year":lambda x: 2017 - max(x), "Action_Type_Code":"count", "geometry":"first","address2":"first", "own_occupd":"first"}



In [ ]:
#Aggregate the data onto the location ID field

commondf = negative_master.groupby("Location_ID").agg(aggro4)

In [ ]:
#Change the column names
commondf.columns = ['com_case', 'geometry', 'address', 'negCodecnt', 'last_case', 'own_occup']

In [ ]:
#Drop Geometry column
commondf.drop('geometry', axis=1, inplace=True)

In [ ]:
#Reset Index
commondf.reset_index(inplace=True)

In [ ]:
#Merge code violations with the housing geocode file
code_viol_agg_final = pd.merge(commondf, all_geocode.drop_duplicates(subset=['address']),\
                                how = "left", left_on = "address",\
                             right_on = "address")

In [ ]:
#Name the file
fname_codviolagg = "DD_code_viol_agg.csv"

In [ ]:
#Create the output path
code_viol_agg_finalOUTcsv = os.path.join(uppath, dpath, outpath, fname_codviolagg)

In [ ]:
#Export to csv
code_viol_agg_final.to_csv(code_viol_agg_finalOUTcsv)

In [ ]:
##End of code